In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import pandas as pd
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        # Set padding token if it doesn't exist
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors='pt'
        )

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = item['input_ids'].clone()
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
def finetuning_data():
    texts = [
        "Question: What is machine learning? Answer: Machine learning is a branch of AI that enables systems to learn from data.",
        "Question: Whar is neural networks? Answer: Neural networks are computing systems inspired by biological neural networks.",
        "Question: What is deep learning? Answer: Deep learning is a subset of machine learning that uses neural networks with multiple hidden layers.",
        "Question: What is computer vision? Answer: Computer vision is a field of AI that enables computers to interpret and understand the visual world.",
        "Question: What is natural language processing? Answer: It is a field of AI that enables computers to understand, interpret, and utilize human language."
    ]
    return texts

In [ ]:
def fine_tune_model(model_name="gpt2", output_dir="./fine_tuned_model"):
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Set padding token for both tokenizer and model
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    # Prepare dataset
    texts = finetuning_data()
    dataset = CustomDataset(texts, tokenizer)

    # Define training arguments
    training_args = TrainingArguments(output_dir=output_dir,num_train_epochs=3,
        per_device_train_batch_size=4, save_steps=100, save_total_limit=2,
        logging_steps=10, learning_rate=2e-5, weight_decay=0.01,
        report_to=["none"], logging_dir="./logs")

    # Initialize trainer
    trainer = Trainer(model=model, args=training_args, train_dataset=dataset)

    # Start training
    trainer.train()

    # Save the model and tokenizer
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)

    return output_dir

In [ ]:
def generate_response(prompt, model_path, max_length=30):
    """
    Generate a response using the fine-tuned model.
    """
    # Load the fine-tuned model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)

    # Set padding token if it doesn't exist
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    # Prepare input
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(inputs.input_ids, max_length=max_length,
            num_return_sequences=1, pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id, do_sample=True, temperature=0.7)

    # Decode and return response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
# Fine-tune and get model path of the fine-tuned the model
model_path = fine_tune_model()

# Example usage of the fine-tuned model
prompt = "Question: What is reinforcement learning?"
response = generate_response(prompt, model_path)
print(f"Generated response: {response}")

<ipython-input-1-d3404f51b0d6>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


Generated response: Question: What is reinforcement learning?

A reinforcement learning is a process that uses information from your environment to help you learn and improve. It is
